In [ ]:
import random
import time
import Statistics as st

# Inject noise into the data part of the frame
def noisychannel(frame):
    frame_list = list(frame[-st.CRC_SIZE-st.DATA_SIZE:-st.CRC_SIZE])
    # Randomly select positions to flip bits
    positions = random.sample(range(len(frame_list)), random.randint(0, len(frame_list) - 1))
    for index in positions:
        frame_list[index] = '1' if frame_list[index] == '0' else '0'
    
    # Rebuild the frame with the modified data
    return frame[:st.N_SIZE + st.LENGTH_SIZE] + ''.join(frame_list) + frame[-st.CRC_SIZE:]

# Introduce random delay up to 5 seconds
def delay():
    time.sleep(random.randint(0, 5))

# XOR operation between two binary strings
def xor(a, b):
    return ''.join('0' if a[i] == b[i] else '1' for i in range(len(b)))

# Perform binary division for CRC calculation
def binary_division(dividend, divisor):
    rem = dividend[:len(divisor)]
    for i in range(len(divisor), len(dividend)):
        rem = xor(rem, divisor)[1:] if rem[0] == '1' else rem[1:]
        rem += dividend[i]
    return rem

# CRC-4 ITU calculation
def crc4itu(frame):
    return binary_division(frame + '0000', "10011")

# Create a frame with given nth frame number and data
def makeFrame(n, data):
    frame = f"{str(n).zfill(st.N_SIZE)}{str(len(data)).zfill(st.LENGTH_SIZE)}{data.zfill(st.DATA_SIZE)}{crc4itu(data).zfill(st.CRC_SIZE)}"
    return frame

# Extract details from received frame
def receiveFrame(frame):
    n = int(frame[:st.N_SIZE])
    l = int(frame[st.N_SIZE:st.N_SIZE + st.LENGTH_SIZE])
    data = frame[-st.CRC_SIZE-l:-st.CRC_SIZE]
    crc = frame[-st.CRC_SIZE:] if data != "q" else ""
    return n, l, data, crc
